In [16]:
import requests 
import time
from tqdm import tqdm
import json
import pandas as pd
import os
from fake_useragent import UserAgent

In [17]:
collection = "CRCHAMPS-d0265d"
chunk_size = 50
sleep_time = 0.4

In [18]:
with open(f'data/{collection}_nfts.json', 'r') as f:
    nfts = json.load(f)

In [19]:
if f'{collection}_transactions.json' in os.listdir():
    with open(f'{collection}_transactions.json', 'r') as f:
        transactions_data = json.load(f)
else:
    transactions_data = {}

In [22]:
for identifier, nft in tqdm(nfts.items()):
    if identifier in transactions_data and transactions_data[identifier]['count'] == len(transactions_data[identifier]['transactions']):
        continue
    transactions_data[identifier] = {
        'count': -1,
        'transactions': []
    }
    time.sleep(sleep_time)
    url = f'https://api.elrond.com/transfers/count?token={identifier}'
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    count = requests.get(url, headers=headers).json()
    transactions_data[identifier]['count'] = count
    for index in range(0, count, chunk_size):
        time.sleep(sleep_time)
        url = f'https://api.elrond.com/transfers?from={index}&size={chunk_size}&token={identifier}&withScamInfo=true&withUsername=true&withBlockInfo=true'
        user_agent = UserAgent().random
        headers = {'User-Agent': user_agent}
        transactions_data[identifier]['transactions'] += requests.get(url, headers=headers).json()

 32%|███▏      | 190/600 [01:41<03:39,  1.86it/s] 


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [23]:
transactions_data['CRCHAMPS-d0265d-01']

{'count': 7,
 'transactions': [{'txHash': '591afa8bf788ad02bbc6213e656cdc0bf63d639ed52356932730dace263ac084',
   'gasLimit': 1000000,
   'gasPrice': 1000000000,
   'gasUsed': 439950,
   'miniBlockHash': '1aeb338b170a5485c67af3d1f058c0fddd5098d127ad6655acfd232d20d41276',
   'nonce': 12,
   'receiver': 'erd19tcl5574296aky2hmyppzk8a3e8v9vhuhy6ycz9xwm5lxcne936q09r5gw',
   'receiverShard': 0,
   'round': 19523413,
   'sender': 'erd19tcl5574296aky2hmyppzk8a3e8v9vhuhy6ycz9xwm5lxcne936q09r5gw',
   'senderShard': 0,
   'signature': '9b3abb15a95d0edd189d87bd2d3d60f0c21ec8cea00e3df42c9c95859a40072ac5c3c020afdd88e5a9e346e599dd8f20950286635749e434bec9f30e2c175209',
   'status': 'success',
   'value': '0',
   'fee': '227644500000000',
   'timestamp': 1713258078,
   'data': 'RVNEVE5GVFRyYW5zZmVyQDQzNTI0MzQ4NDE0ZDUwNTMyZDY0MzAzMjM2MzU2NEAwMUAwMUBmM2I1ZWUyMzk3ZTA1ZDAyMmEwY2IxZTM5ZDJiYjYzMmJhMGUyMDM0MGQ4ZDJjODBmOGY3NmI4MjE0NWVhZjgw',
   'function': 'ESDTNFTTransfer',
   'action': {'category': 'esdtNft',

In [15]:
with open(f'{collection}_transactions.json', 'w') as f:
    json.dump(transactions_data, f, indent=4)

In [16]:
c = 0 
for identifier in transactions_data:
    c += transactions_data[identifier]['count']
c

441176